In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/ND_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,5206477102,ND,"322 Demers Ave Suite 500, GRAND FORKS, ND",47.924918,-97.031593,3.803501e+14
1,2976507101,ND,"1600 N 48TH ST, GRAND FORKS, ND",47.936733,-97.098896,3.803501e+14
2,5041237003,ND,"7525 RED RIVER RD, WAHPETON, ND",46.324042,-96.615776,3.807797e+14
3,4487847009,ND,"3520 E. Century Avenue, BISMARCK, ND",46.838560,-100.735451,3.801501e+14
4,2579588901,ND,"1202 Page Dr S, Fargo, ND",46.863174,-96.827143,3.801700e+14


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,5206477102,ND,"322 Demers Ave Suite 500, GRAND FORKS, ND",47.924918,-97.031593,3.803501e+14
1,2976507101,ND,"1600 N 48TH ST, GRAND FORKS, ND",47.936733,-97.098896,3.803501e+14
2,5041237003,ND,"7525 RED RIVER RD, WAHPETON, ND",46.324042,-96.615776,3.807797e+14
3,4487847009,ND,"3520 E. Century Avenue, BISMARCK, ND",46.838560,-100.735451,3.801501e+14
4,2579588901,ND,"1202 Page Dr S, Fargo, ND",46.863174,-96.827143,3.801700e+14


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
688,4371027008,ND,"202 6TH AVE, FLASHER, ND",40.726624,-74.002917,NaN
692,6225057007,ND,"705 13th Ave S, WEST FARGO, ND",36.150132,-86.786585,NaN
930,2088118307,ND,"10199 Lanesboro Rd, Scranton, ND",45.852712,-103.252859,NaN
1035,4443287005,ND,"2355 Highway 10, MANDAN, ND",43.900656,-85.135021,NaN
1142,7274018602,ND,"11801 Highway 2 and 52 W, Burlington, ND",47.597442,-120.649613,NaN
2469,7773717010,ND,"369 Hwy 81 SE PO BOX 100, HILLSBORO, ND",33.437031,-97.762368,NaN
2492,8532788608,ND,"369 HWY 81 SE, Hillsboro, ND",33.437031,-97.762368,NaN
2600,6843438007,ND,"100 S 4TH ST, SUITE 104, FARGO, ND",46.853305,-96.773614,NaN
3205,2189357105,ND,"103 9TH ST S, COOPERSTOWN, ND",46.873599,-96.766589,NaN
3214,9920748603,ND,"320 Main St, Arthur, ND",33.743250,-118.103167,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('38')] #ND
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/10 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

10


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_13743/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
688,4371027008,ND,"202 6TH AVE, FLASHER, ND",40.726624,-74.002917,[]
692,6225057007,ND,"705 13th Ave S, WEST FARGO, ND",36.150132,-86.786585,[]
930,2088118307,ND,"10199 Lanesboro Rd, Scranton, ND",45.852712,-103.252859,[]
1035,4443287005,ND,"2355 Highway 10, MANDAN, ND",43.900656,-85.135021,[]
1142,7274018602,ND,"11801 Highway 2 and 52 W, Burlington, ND",47.597442,-120.649613,[]
2469,7773717010,ND,"369 Hwy 81 SE PO BOX 100, HILLSBORO, ND",33.437031,-97.762368,[]
2492,8532788608,ND,"369 HWY 81 SE, Hillsboro, ND",33.437031,-97.762368,[]
2600,6843438007,ND,"100 S 4TH ST, SUITE 104, FARGO, ND",46.853305,-96.773614,[]
3205,2189357105,ND,"103 9TH ST S, COOPERSTOWN, ND",46.873599,-96.766589,[]
3214,9920748603,ND,"320 Main St, Arthur, ND",33.743250,-118.103167,[]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_13743/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
688,4371027008,ND,"202 6TH AVE, FLASHER, ND",40.726624,-74.002917,NaN
692,6225057007,ND,"705 13th Ave S, WEST FARGO, ND",36.150132,-86.786585,NaN
930,2088118307,ND,"10199 Lanesboro Rd, Scranton, ND",45.852712,-103.252859,NaN
1035,4443287005,ND,"2355 Highway 10, MANDAN, ND",43.900656,-85.135021,NaN
1142,7274018602,ND,"11801 Highway 2 and 52 W, Burlington, ND",47.597442,-120.649613,NaN
2469,7773717010,ND,"369 Hwy 81 SE PO BOX 100, HILLSBORO, ND",33.437031,-97.762368,NaN
2492,8532788608,ND,"369 HWY 81 SE, Hillsboro, ND",33.437031,-97.762368,NaN
2600,6843438007,ND,"100 S 4TH ST, SUITE 104, FARGO, ND",46.853305,-96.773614,NaN
3205,2189357105,ND,"103 9TH ST S, COOPERSTOWN, ND",46.873599,-96.766589,NaN
3214,9920748603,ND,"320 Main St, Arthur, ND",33.743250,-118.103167,NaN


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
688,4371027008,ND,"202 6TH AVE, FLASHER, ND",40.726624,-74.002917,NaN
692,6225057007,ND,"705 13th Ave S, WEST FARGO, ND",36.150132,-86.786585,NaN
930,2088118307,ND,"10199 Lanesboro Rd, Scranton, ND",45.852712,-103.252859,NaN
1035,4443287005,ND,"2355 Highway 10, MANDAN, ND",43.900656,-85.135021,NaN
1142,7274018602,ND,"11801 Highway 2 and 52 W, Burlington, ND",47.597442,-120.649613,NaN
2469,7773717010,ND,"369 Hwy 81 SE PO BOX 100, HILLSBORO, ND",33.437031,-97.762368,NaN
2492,8532788608,ND,"369 HWY 81 SE, Hillsboro, ND",33.437031,-97.762368,NaN
2600,6843438007,ND,"100 S 4TH ST, SUITE 104, FARGO, ND",46.853305,-96.773614,NaN
3205,2189357105,ND,"103 9TH ST S, COOPERSTOWN, ND",46.873599,-96.766589,NaN
3214,9920748603,ND,"320 Main St, Arthur, ND",33.743250,-118.103167,NaN


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
688,4371027008,NaN
692,6225057007,NaN
930,2088118307,NaN
1035,4443287005,NaN
1142,7274018602,NaN
2469,7773717010,NaN
2492,8532788608,NaN
2600,6843438007,NaN
3205,2189357105,NaN
3214,9920748603,NaN


Read in base data

In [14]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/ND_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,544237,5206477102,ND,"322 Demers Ave Suite 500, GRAND FORKS, ND",47.924918,-97.031593,3.803501e+14
1,544238,2976507101,ND,"1600 N 48TH ST, GRAND FORKS, ND",47.936733,-97.098896,3.803501e+14
2,544239,5041237003,ND,"7525 RED RIVER RD, WAHPETON, ND",46.324042,-96.615776,3.807797e+14
3,544240,4487847009,ND,"3520 E. Century Avenue, BISMARCK, ND",46.838560,-100.735451,3.801501e+14
4,544241,2579588901,ND,"1202 Page Dr S, Fargo, ND",46.863174,-96.827143,3.801700e+14


Merge with MainFrame

In [15]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,544237,5206477102,ND,"322 Demers Ave Suite 500, GRAND FORKS, ND",47.924918,-97.031593,3.803501e+14,NaN
1,544238,2976507101,ND,"1600 N 48TH ST, GRAND FORKS, ND",47.936733,-97.098896,3.803501e+14,NaN
2,544239,5041237003,ND,"7525 RED RIVER RD, WAHPETON, ND",46.324042,-96.615776,3.807797e+14,NaN
3,544240,4487847009,ND,"3520 E. Century Avenue, BISMARCK, ND",46.838560,-100.735451,3.801501e+14,NaN
4,544241,2579588901,ND,"1202 Page Dr S, Fargo, ND",46.863174,-96.827143,3.801700e+14,NaN
...,...,...,...,...,...,...,...,...
3217,547454,2849968506,ND,"3913 106th Ave SE, Sanborn, ND",46.848916,-98.228272,3.800397e+14,NaN
3218,547455,3026678408,ND,"1218 S Washington St, Grand Forks, ND",47.909497,-97.046115,3.803501e+14,NaN
3219,547456,3175408401,ND,"5458 134th Ave NW, Williston, ND",48.205512,-103.580819,3.810595e+14,NaN
3220,547457,3860267409,ND,"905 3RD AVE SW, WATFORD CITY, ND",47.796467,-103.295171,3.805396e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [16]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,544237,5206477102,ND,"322 Demers Ave Suite 500, GRAND FORKS, ND",47.924918,-97.031593,3.803501e+14,NaN,3.803501e+14
1,544238,2976507101,ND,"1600 N 48TH ST, GRAND FORKS, ND",47.936733,-97.098896,3.803501e+14,NaN,3.803501e+14
2,544239,5041237003,ND,"7525 RED RIVER RD, WAHPETON, ND",46.324042,-96.615776,3.807797e+14,NaN,3.807797e+14
3,544240,4487847009,ND,"3520 E. Century Avenue, BISMARCK, ND",46.838560,-100.735451,3.801501e+14,NaN,3.801501e+14
4,544241,2579588901,ND,"1202 Page Dr S, Fargo, ND",46.863174,-96.827143,3.801700e+14,NaN,3.801700e+14


Drop Extra FIPS Columns

In [17]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,5206477102,ND,"322 Demers Ave Suite 500, GRAND FORKS, ND",47.924918,-97.031593,3.803501e+14
1,2976507101,ND,"1600 N 48TH ST, GRAND FORKS, ND",47.936733,-97.098896,3.803501e+14
2,5041237003,ND,"7525 RED RIVER RD, WAHPETON, ND",46.324042,-96.615776,3.807797e+14
3,4487847009,ND,"3520 E. Century Avenue, BISMARCK, ND",46.838560,-100.735451,3.801501e+14
4,2579588901,ND,"1202 Page Dr S, Fargo, ND",46.863174,-96.827143,3.801700e+14
...,...,...,...,...,...,...
3217,2849968506,ND,"3913 106th Ave SE, Sanborn, ND",46.848916,-98.228272,3.800397e+14
3218,3026678408,ND,"1218 S Washington St, Grand Forks, ND",47.909497,-97.046115,3.803501e+14
3219,3175408401,ND,"5458 134th Ave NW, Williston, ND",48.205512,-103.580819,3.810595e+14
3220,3860267409,ND,"905 3RD AVE SW, WATFORD CITY, ND",47.796467,-103.295171,3.805396e+14


In [18]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
688,4371027008,ND,"202 6TH AVE, FLASHER, ND",40.726624,-74.002917,NaN
692,6225057007,ND,"705 13th Ave S, WEST FARGO, ND",36.150132,-86.786585,NaN
930,2088118307,ND,"10199 Lanesboro Rd, Scranton, ND",45.852712,-103.252859,NaN
1035,4443287005,ND,"2355 Highway 10, MANDAN, ND",43.900656,-85.135021,NaN
1142,7274018602,ND,"11801 Highway 2 and 52 W, Burlington, ND",47.597442,-120.649613,NaN
2469,7773717010,ND,"369 Hwy 81 SE PO BOX 100, HILLSBORO, ND",33.437031,-97.762368,NaN
2492,8532788608,ND,"369 HWY 81 SE, Hillsboro, ND",33.437031,-97.762368,NaN
2600,6843438007,ND,"100 S 4TH ST, SUITE 104, FARGO, ND",46.853305,-96.773614,NaN
3205,2189357105,ND,"103 9TH ST S, COOPERSTOWN, ND",46.873599,-96.766589,NaN
3214,9920748603,ND,"320 Main St, Arthur, ND",33.743250,-118.103167,NaN


Convert to File

In [19]:
fips_ND = fips_merge_drop

In [20]:
fips_ND.to_csv("../../../data/state_data/geo/geo_fips/23/ND_fips_scraped.csv") 